<a href="https://colab.research.google.com/github/doctorpeppercokebear/DEEPLEANNING/blob/tensorflow/%5B%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8%5DCh01_%E1%84%8C%E1%85%A6%E1%84%8C%E1%85%A9%E1%84%80%E1%85%A9%E1%86%BC%E1%84%8C%E1%85%A5%E1%86%BC_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%8B%E1%85%A3_04_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8_%E1%84%92%E1%85%AA%E1%86%AB%E1%84%80%E1%85%A7%E1%86%BC_%E1%84%8C%E1%85%AE%E1%86%AB%E1%84%87%E1%85%B5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 환경 준비

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [ ]:
PATH_DEFECT = 'dataset/Defect_images/'
PATH_MASK = 'dataset/Mask_images/'
PATH_NODEFECT = 'dataset/NODefect_images/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ./drive/MyDrive/deeplearning/NODefect_images.7z ./

In [ ]:
!cp ./drive/MyDrive/deeplearning/Mask_images.7z ./

In [ ]:
!cp ./drive/MyDrive/deeplearning/Defect_images.7z ./

In [ ]:
! 7zr x ./NODefect_images.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 98321827 bytes (94 MiB)

Extracting archive: ./NODefect_images.7z
--
Path = ./NODefect_images.7z
Type = 7z
Physical Size = 98321827
Headers Size = 1749
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     40% 56 - NODefect_images/2311517-195063u/0009_000_04.png                                                          43% 61 - NODefect_images/2311517-195063u/0014_000_04.png                                                          45% 62 - NODefect_images/2311517-195063u/001

In [ ]:
! 7zr x ./Mask_images.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 88311 bytes (87 KiB)

Extracting archive: ./Mask_images.7z
--
Path = ./Mask_images.7z
Type = 7z
Physical Size = 88311
Headers Size = 1496
Method = LZMA2:18
Solid = +
Blocks = 1

  0%    Everything is Ok

Folders: 1
Files: 107
Size:       216186
Compressed: 88311


In [ ]:
! 7zr x ./Defect_images.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 77676758 bytes (75 MiB)

Extracting archive: ./Defect_images.7z
--
Path = ./Defect_images.7z
Type = 7z
Physical Size = 77676758
Headers Size = 1620
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     74% 74 - Defect_images/0077_022_03.png                                       Everything is Ok

Folders: 1
Files: 106
Size:       77674496
Compressed: 77676758


In [ ]:
!mkdir dataset

In [ ]:
!mv ./Defect_images/ ./dataset/

In [ ]:
!mv ./NODefect_images/ ./dataset/

In [ ]:
!mv ./Mask_images/ ./dataset/

In [ ]:
!ls -al ./dataset      # 파일 확인

total 20
drwxr-xr-x 5 root root 4096 Jul 14 04:55 .
drwxr-xr-x 1 root root 4096 Jul 14 04:55 ..
drwx------ 2 root root 4096 Jul  5  2019 Defect_images
drwx------ 2 root root 4096 Jul  5  2019 Mask_images
drwx------ 9 root root 4096 Jul  5  2019 NODefect_images


In [ ]:
PATH_DEFECT = 'dataset/Defect_images/'
PATH_MASK = 'dataset/Mask_images/'
PATH_NODEFECT = 'dataset/NODefect_images/'

## Dataset 불러오기

In [ ]:
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('/')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('/')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

## 첫 발송 데이터 생성

In [ ]:
# The first dataset given
if os.path.exists('dataset/1') is False:
    os.mkdir('dataset/1')
for file_name in pass_list + defect_list:
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        shutil.copy2(file_name, 'dataset/1/' + barcode + '.png')

## 두번째 데이터 생성

In [ ]:
# The second dataset
if os.path.exists('dataset/2') is False:
    os.mkdir('dataset/2')
if os.path.exists('dataset/2/OK') is False:
    os.mkdir('dataset/2/OK')
if os.path.exists('dataset/2/FAIL') is False:
    os.mkdir('dataset/2/FAIL')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 2:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/2/OK/%04d.png' % idx, patch)
            idx += 1

patch_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_list.append(patch)

random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
    cv2.imwrite('dataset/2/FAIL/%04d.png' % idx, patch)

## 세번째 데이터 생성

In [ ]:
# The third dataset
if os.path.exists('dataset/3') is False:
    os.mkdir('dataset/3')
if os.path.exists('dataset/3/OK') is False:
    os.mkdir('dataset/3/OK')
if os.path.exists('dataset/3/FAIL') is False:
    os.mkdir('dataset/3/FAIL')
if os.path.exists('dataset/3/MASK') is False:
    os.mkdir('dataset/3/MASK')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 3:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/3/OK/%04d.png' % idx, patch)
            idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('dataset/3/FAIL/%04d.png' % idx, patch)
    cv2.imwrite('dataset/3/MASK/%04d.png' % idx, patch_d)

## 실전 데이터 생성

In [ ]:
# The test dataset
if os.path.exists('dataset/input_data') is False:
    os.mkdir('dataset/input_data')
if os.path.exists('dataset/output_csv') is False:
    os.mkdir('dataset/output_csv')

idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 5:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/input_data/ok_%04d.png' % idx, patch)
            idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('dataset/input_data/fail_%04d.png' % idx, patch)

In [ ]:
! 7zr a dataset.7z ./dataset


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  ./             21 folders, 5755 files, 856563573 bytes (817 MiB)

Creating archive: dataset.7z

Items to compress: 5776

  0%      0% 1 + dataset/1/2EtCY2Yl9NfS9yQt.png                                         0% 2 + dataset/1/4cN2wUQosf1UV2zd.png                                         0% 3 + dataset/1/6Di3LfRYvcCCgWbL.png                                         0% 4 + dataset/1/73UZISA1gMSMcvTx.png                                       

In [ ]:
!cp ./dataset.7z ./drive/MyDrive/